In [ ]:
import matplotlib.pylab as plt
# from matplotlib.backends.backend_pdf import PdfPages
# from scipy.spatial.distance import correlation as scorr
# from copulalib.copulalib import Copula
# from scipy.optimize import minimize
# from sklearn import manifold
import numpy as np
# import os.path
# import glob
# from utils import *
from pyemd import emd
# %pylab inline
# import re

# import pyemma.plots as mplt

In [ ]:
def build_sigrature(x_all, y_all, z_all):
    signatures = []
    i = 0
    for u1 in x_all[1:]:
        j = 0
        for u2 in y_all[1:]:
            signatures.append([np.array([u1,u2]),z_all[i,j]])
            j += 1
        i += 1
                        
    return signatures

In [ ]:
def build_dist_mat(signatures):
    N = len(signatures)
    dist_mat = np.zeros((N,N))
    for i in range(0,len(signatures)):
        for j in range(i,len(signatures)):
            v1 = signatures[i][0]
            v2 = signatures[j][0]
            dist_mat[i,j] = np.linalg.norm(v1-v2)
            dist_mat[j,i] = dist_mat[i,j]
    
    return dist_mat

### The "build_emd_array" function computes EMD values for all provided densities with respect to the first provided density. Thereby one can easily compute time resolved EMD comparions.

In [1]:
# The build_emd_array function computes EMD values for all provided densities with respect to 
# the first provided density

def build_emd_array(list_of_densities, list_of_labels, x_all, y_all):

    all_val = list_of_densities
    all_val_names = list_of_labels
    to_plot = []
    labels = []

    print("Caution: If number of bins is > 30 calculations can be to long!")
    signatures = build_sigrature(x_all,y_all, all_val[0])
    dist_mat = build_dist_mat(signatures)
    
    for i in range(1,len(all_val)):
        tmp = []
        if len(all_val[i].ravel()) > 100*100: 
            print("Histogramm is to big, reduce number of bins to max 100")
            return []
        else: 
            sig_S = all_val[0].ravel()
            sig_G = all_val[i].ravel()
            dist_SG = emd(sig_S,sig_G,dist_mat)
            to_plot.append(dist_SG)
        labels.append(str(all_val_names[0])+"_"+str(all_val_names[i]))
    
    labels = np.array(labels)
    to_plot = np.array(to_plot)
    
    centers = np.linspace(0, len(to_plot), len(to_plot))


    f = plt.figure(figsize=(len(to_plot)+3, 4), dpi=100)
    plt.xlim(-0.5, len(to_plot)+0.5)
    plt.xticks(centers, labels)
    plt.tick_params(axis='both', which='major', labelsize=14)

    plt.title(r"Earth Mover Distance", fontsize = 20)
    plt.plot(centers, 1-to_plot/max(to_plot), linewidth =3.5, marker = 'o')
    
    return to_plot

In [ ]:
lst_all_loaded_projections = [np.load('')]  # the first histogram is the one all other provided histograms 
                                            # are compared to (i.e. the CG projection)
    
lst_of_projections_to_compare = []          # provide here all histograms that should be compared to the first one

for projection in lst_of_projections_to_compare:
    lst_all_loaded_projections.append(np.load(projection))

In [ ]:
all_val_names = [""]   ### List of names of the projections provided above (be aware of the right order!)

In [ ]:
all_prj = np.vstack(lst_all_loaded_projections)
print(all_prj.shape)

In [ ]:
z_all,x_all,y_all = np.histogram2d(all_prj[:,0],all_prj[:,1], bins=40)   ### Number of bins can be varied

F_all = -np.log(z_all) ### compute free energies

plt.figure(figsize=(8, 6), dpi=100)  ### contour plot
extent_all = [x_all[0], x_all[-1], y_all[0], y_all[-1]]
plt.contourf(F_all.T, 50, cmap=plt.cm.hot, extent=extent_all)

In [ ]:
all_histos = []
for prj in lst_all_loaded_projections:
    z_tmp,x_tmp,y_tmp = np.histogram2d(prj[:,0],prj[:,1], bins=(x_all, y_all))
    all_histos.append(z_tmp/z_tmp.sum())

In [ ]:
emd_dists = build_emd_array(all_histos, all_val_names, x_all, y_all)